In [6]:
### Adjusting the disposable_incomes 

In [11]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import numpy as np
import scipy.stats as stats

class WealthAgent(Agent):
    def __init__(self, unique_id, model, disposable_income, dwelling, technology):
        super().__init__(unique_id, model)
        self.disposable_income = disposable_income
        self.dwelling = dwelling
        self.technology = technology

    @property
    def energy_cost(self):
        """Calculate and return the current energy cost for the agent."""
        return self.model.energy_price * (self.dwelling + self.technology)

    @property
    def inability(self):
        """Check if the agent has inability to keep home adequately warm."""
        return self.energy_cost > self.model.inability_threshold * self.disposable_income

    def step(self):
        """Define the agent's behavior in a single step of the model."""
        pass


class WealthModel(Model):
    def __init__(self, N, median_income, gini, rate, energy_price_x, share_fuel_x, energy_price_y, share_fuel_y, inability_target):
        self.num_agents = N
        self.median_income = median_income
        self.gini = gini
        self.rate = rate
        self.energy_price_x = energy_price_x
        self.share_fuel_x = share_fuel_x
        self.energy_price_y = energy_price_y
        self.share_fuel_y = share_fuel_y
        self.inability_target = inability_target
        self.inability_threshold = 0.1  # Initialize inability_threshold to 0.1

        self.schedule = RandomActivation(self)
        self.datacollector = DataCollector(
            agent_reporters={"Inability": "inability",
                             "DisposableIncome": "disposable_income"}
        )

        # Generate disposable incomes according to a gamma distribution shaped by the Gini coefficient
        incomes = self.generate_income_distribution(self.num_agents, self.median_income, self.gini)

        # Create agents
        for i in range(self.num_agents):
            # Initialize dwelling and technology as zeros for simplicity. 
            # You can add the specific logic to initialize these attributes later.
            a = WealthAgent(i, self, disposable_income=incomes[i], dwelling=0, technology=0)
            self.schedule.add(a)

    @property
    def energy_price(self):
        """Calculate and return the current energy price in the model."""
        return self.energy_price_x * self.share_fuel_x + self.energy_price_y * self.share_fuel_y
    

    def scale_incomes(self, factor):
        """Scale all disposable incomes by the given factor."""
        for agent in self.schedule.agents:
            agent.disposable_income *= factor

    @staticmethod
    def generate_income_distribution(num_people, median_income, gini):
        alpha = (gini + 1) / (2 - gini)
        incomes = stats.gamma.rvs(alpha, scale=median_income/alpha, size=num_people)
        return incomes

    def step(self):
        """Advance the model by one step."""
        self.schedule.step()
        self.datacollector.collect(self)

        # Set initial bounds for binary search
        low, high = 0.5, 1.5  # Assume that the incomes need to be scaled by a factor in the range [0.5, 1.5]

        # Binary search for the scaling factor that will result in an inability share close to the target
        for _ in range(100):  # Limit the number of iterations to prevent infinite loops
            mid = (low + high) / 2

            # Scale disposable incomes by mid
            for agent in self.schedule.agents:
                agent.disposable_income *= mid

            # Recalculate inability_share
            inability_share = sum(agent.inability for agent in self.schedule.agents) / self.num_agents

            # Check if inability share is close enough to the target
            if abs(inability_share - self.inability_target) <= 0.01:
                break

            # Update low or high based on whether inability share is lower or higher than the target, respectively
            if inability_share < self.inability_target:
                low = mid
            else:
                high = mid


# Instantiate the model with hypothetical parameters and run for a certain number of steps
model = WealthModel(100, 60000, 0.3, [0.01, 0.05], 1, 0.5, 1, 0.5, 0.2)

for i in range(100):
    model.step()

# Collect data
agent_data = model.datacollector.get_agent_vars_dataframe()


C:\Users\novak\AppData\Local\Temp\ipykernel_19080\1246809703.py:89: RuntimeWarning: overflow encountered in double_scalars
  agent.disposable_income *= mid


In [12]:
agent_data.Inability.tail(99).sum()

0